# News classification using BERT

## Import libraries

In [5]:
import math
import os

import numpy as np
import pandas as pd
import lightning as L
import torch
import torch.nn.functional as F
import torchtext.datasets as td
from lightning.pytorch.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    LearningRateMonitor,
)
from lightning.pytorch.cli import LightningCLI
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchdata.datapipes.iter import IterDataPipe
from torchtext.data.functional import to_map_style_dataset
from torchtext.datasets import AG_NEWS
from transformers import BertModel, BertTokenizer, AdamW

import mlflow.pytorch

In [6]:
def get_20newsgroups(num_samples):
    categories = ["alt.atheism", "talk.religion.misc", "comp.graphics", "sci.space"]
    X, y = fetch_20newsgroups(subset="train", categories=categories, return_X_y=True)
    return pd.DataFrame(data=X, columns=["description"]).assign(label=y).sample(n=num_samples)

twenty_groups_news = get_20newsgroups(1000)
twenty_groups_news.head()

,description,label
1767,From: kmr4@po.CWRU.edu (Keith M. Ryan)\nSubjec...,0
277,"From: ldo@waikato.ac.nz (Lawrence D'Oliveiro, ...",1
768,From: nyeda@cnsvax.uwec.edu (David Nye)\nSubje...,0
654,From: keith@cco.caltech.edu (Keith Allan Schne...,0
1673,From: ajjb@adam4.bnsc.rl.ac.uk (Andrew Broderi...,2


In [8]:
twenty_groups_news.description.values[0]

'From: kmr4@po.CWRU.edu (Keith M. Ryan)\nSubject: Re: "Cruel" (was Re: <Political Atheists?)\nOrganization: Case Western Reserve University\nLines: 33\nNNTP-Posting-Host: b64635.student.cwru.edu\n\nIn article <1ql8mdINN674@gap.caltech.edu> keith@cco.caltech.edu (Keith Allan Schneider) writes:\n\n>>>This whole thread started because of a discussion about whether\n>>>or not the death penalty constituted cruel punishment, which is forbidden\n>>>by the US Constitution.\n>>Yes, but they didn\'t say what they meant by "cruel", which is why\n>>a) you have the Supreme Court, and b) it makes no sense to refer\n>>to the Constitution, which is quite silent on the meaning of the\n>>word "cruel".\n>\n>They spent quite a bit of time on the wording of the Constitution.  They\n>picked words whose meanings implied the intent.  We have already looked\n>in the dictionary to define the word.  Isn\'t this sufficient?\n\n\tWe only need to ask the question: what did the founding fathers \nconsider cruel and 

In [9]:
twenty_groups_news.label.value_counts()

2    297
1    281
0    226
3    196
Name: label, dtype: int64

In [7]:
def get_ag_news(num_samples):
    # reading the input
    AG_NEWS(".", split=("train", "test"))
    train_csv_path = "data/AG_NEWS/train.csv"
    return (
        pd.read_csv(train_csv_path, usecols=[0, 2], names=["label", "description"])
        .assign(label=lambda df: df["label"] - 1)  # make labels zero-based
        .sample(n=num_samples)
    )

ag_news = get_ag_news(1000)

FileNotFoundError: [Errno 2] No such file or directory: 'data/AG_NEWS/train.csv'